In [1]:
from src.model.nli_models import *
from src.model.novelty_models import *
from src.defaults import *

def load_novelty_model(_id):
    # load model data 
    check_model(_id)
    def load_model_data(_id):
        model_path = os.path.join("./results/", _id, "model.pt")
        model_data = torch.load(model_path)
        return model_data
    field = load_field(_id)
    model_data = load_model_data(_id)
    encoder_id = model_data["options"]["load_nli"]
    check_model(encoder_id)

    def load_encoder(enc_data):
        if enc_data["options"].get("attention_layer_param", 0) == 0:
            enc_data["options"]["use_glove"] = False
            model = bilstm_snli(enc_data["options"])
        elif enc_data["options"].get("r", 0) == 0:
            enc_data["options"]["use_glove"] = False
            model = attn_bilstm_snli(enc_data["options"])
        else:
            enc_data["options"]["use_glove"] = False
            model = struc_attn_snli(enc_data["options"])
        model.load_state_dict(enc_data["model_dict"])
        return model
    
    enc_data = load_encoder_data(encoder_id)
    encoder = load_encoder(enc_data).encoder

    model = HAN(model_data["options"],encoder)
    model.load_state_dict(model_data["model_dict"])
    return model,field


In [2]:
from torchtext.data import Example 

def encode_text(text,field):
    ex = Example.fromlist([text],[("text",field)])
    enc = field.process([ex.text])
    return torch.tensor(enc)

In [8]:
model,field = load_novelty_model('NOV-1015')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:61: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1



In [9]:
source = "A balanced exercise routine for an athlete must consists of cardio as well as weighttraining. The way we perform these two exercises is rather important . The researchers believethat one should follow their research which suggests the ideal method for working out. Aworkout should start with some stretching and small dynamic movements to warm up thebody. This should be followed by the weight training or a High Intensity Interval training. Wemust also include a couple minutes of reset between this transition. The can be followed by aproper session of cardio exercises like running or swimming. One should adjust the intensityof this cardio exercises as per their training goal. Researchers also say that a training sessionshould end with a cool-down which includes stretching and relaxation"
s_enc = encode_text(source,field)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning:

Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.

ipykernel_launcher:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [53]:
target = "Research published by a team of sports scientists reveals some interesting in-formation about the recipe for in ideal workout session. A balanced exercise routine for an athlete must consists of cardio as well as weight training. The way we perform these twoexercises is rather important . This should be followed by the weight training or a High In-tensity Interval training. A workout should start with some stretching and small dynamicmovements to warm up the body. The can be followed by a proper session of cardio exerciseslike running or swimming. A couple minutes of rest between these high intensity training isalso considered beneficial. This should be followed by the weight training or a High IntensityInterval training. The fligt from usa to india got delayed.In regex, captures are numbered, but in some implementations, captures can be named. To understand the basics first, let’s see an example in python where capturing is performed in the figure 5.1. In this example, we can see that the input string of ’ac’ is matched."
t_enc = encode_text(target,field)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning:

Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.

ipykernel_launcher:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [54]:
model.eval()
with torch.no_grad():
    pred = F.softmax(model(s_enc,t_enc))

ipykernel_launcher:3: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



In [55]:
pred

tensor([[0.0599, 0.9401]])